In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Mar  21 16:37:31 2019

@author: kushaldeb
"""

import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import classification_report
from tqdm import tqdm_notebook
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
# declaring the constants

batch_size = 64
img_width, img_height = 100, 100
ngpus = 4
nclasses = 2

model_path = 'models-xception/model_weights.h5'

In [3]:
try:
    model = load_model('models-xception/model.h5')
except:
    base_model = Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(nclasses-1, activation='sigmoid')(x)
    pmodel = Model(base_model.input, predictions)
    
    model = multi_gpu_model(pmodel, ngpus)
    model.load_weights('models-xception/model_weights.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
def get_image(path):
    i = img_to_array(load_img(path, target_size=(img_width, img_height)))
    i = np.expand_dims(i, axis=0)
    return i/255.0

In [5]:
prediction = []
actuals = []
image_names = []
probability = []

class_map = {
    0 : '0', 1 : '1'
}

In [6]:
for i in tqdm_notebook(os.listdir('sample_dataset/Validation')):
    for img in tqdm_notebook(os.listdir(f'sample_dataset/Validation/{i}')):
        pred = model.predict(get_image(f'sample_dataset/Validation/{i}/{img}'))[0]
        label = class_map[1] if pred > 0.5 else class_map[0]
        prediction.append(label)
        probability.append(pred)
        actuals.append(i)
        image_names.append(img)
        
data = {
        'image_names' : image_names,
        'actual_prediction' : actuals,
        'predictions' : prediction,
        'probability' : probability
        }

In [7]:
pd.DataFrame(data).to_csv('validation-results-xception.csv', index=False)

x = classification_report(actuals, prediction)
print(x)

              precision    recall  f1-score   support

           0       0.91      0.85      0.88      5000
           1       0.86      0.92      0.89      5000

   micro avg       0.88      0.88      0.88     10000
   macro avg       0.89      0.88      0.88     10000
weighted avg       0.89      0.88      0.88     10000



In [8]:
prediction = []
actuals = []
image_names = []
probability = []

class_map = {
    0 : '0', 1 : '1'
}

In [9]:
for i in tqdm_notebook(os.listdir('sample_dataset/Testing')):
    for img in tqdm_notebook(os.listdir(f'sample_dataset/Testing/{i}')):
        pred = model.predict(get_image(f'sample_dataset/Testing/{i}/{img}'))[0]
        label = class_map[1] if pred > 0.5 else class_map[0]
        prediction.append(label)
        probability.append(pred)
        actuals.append(i)
        image_names.append(img)
        
data = {
        'image_names' : image_names,
        'actual_prediction' : actuals,
        'predictions' : prediction,
        'probability' : probability
        }

In [10]:
pd.DataFrame(data).to_csv('test-results-xception.csv', index=False)

x = classification_report(actuals, prediction)
print(x)

              precision    recall  f1-score   support

           0       0.92      0.85      0.88       500
           1       0.86      0.92      0.89       500

   micro avg       0.89      0.89      0.89      1000
   macro avg       0.89      0.89      0.88      1000
weighted avg       0.89      0.89      0.88      1000

